Collaborative Filtering (User-based Filtering)
References:https://www.youtube.com/watch?v=cxcFi3RDrEw&ab_channel=GrabNGoInfo

#Import libraries

In [1]:
#Google drive connection
from google.colab import drive
drive.mount('/content/drive')

#Data
import pandas as pd
import numpy as np
import scipy.stats

#Visualization
import seaborn as sns

#Similarity
from sklearn.metrics.pairwise import cosine_similarity

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Read Data

In [2]:
path = '/content/drive/MyDrive/Portfolio by Colab/Insurance/Policy Recommendation/insurance_policy_ratings.csv'
data = pd.read_csv(path, index_col='UserID', nrows=1000)
data

,PolicyID1,PolicyID2,PolicyID3,PolicyID4,PolicyID5,PolicyID6,PolicyID7,PolicyID8,PolicyID9,PolicyID10
UserID,,,,,,,,,,
1,4.0,NaN,1.1,NaN,2.1,NaN,NaN,NaN,3.3,NaN
2,4.6,3.3,1.3,NaN,1.4,NaN,NaN,4.2,NaN,3.4
3,2.6,NaN,1.1,NaN,1.5,2.5,NaN,4.0,3.1,2.7
4,3.0,NaN,1.6,2.0,3.5,NaN,3.5,2.3,1.9,NaN
5,NaN,4.4,3.2,2.2,4.4,2.7,2.6,NaN,3.6,4.8
...,...,...,...,...,...,...,...,...,...,...
996,1.7,2.8,4.3,2.1,1.7,NaN,3.5,NaN,NaN,4.7
997,3.0,2.4,5.0,1.6,1.6,NaN,3.7,2.3,1.8,3.6
998,3.5,2.8,NaN,1.9,NaN,NaN,4.0,3.9,2.6,2.8


#Exploratory Data Analysis

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 1 to 1000
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   PolicyID1   665 non-null    float64
 1   PolicyID2   666 non-null    float64
 2   PolicyID3   703 non-null    float64
 3   PolicyID4   670 non-null    float64
 4   PolicyID5   705 non-null    float64
 5   PolicyID6   707 non-null    float64
 6   PolicyID7   712 non-null    float64
 7   PolicyID8   702 non-null    float64
 8   PolicyID9   712 non-null    float64
 9   PolicyID10  707 non-null    float64
dtypes: float64(10)
memory usage: 85.9 KB


In [4]:
data.describe()

,PolicyID1,PolicyID2,PolicyID3,PolicyID4,PolicyID5,PolicyID6,PolicyID7,PolicyID8,PolicyID9,PolicyID10
count,665.000000,666.000000,703.000000,670.000000,705.000000,707.000000,712.000000,702.000000,712.000000,707.000000
mean,3.060150,3.031381,2.927312,2.964627,2.987801,2.958416,2.997612,2.997436,2.964607,3.059406
std,1.185514,1.105988,1.177626,1.127845,1.125161,1.157680,1.126566,1.180691,1.138724,1.179586
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,2.000000,2.100000,1.900000,2.000000,2.000000,1.900000,2.100000,2.000000,2.000000,2.000000
50%,3.100000,3.050000,2.900000,3.000000,2.900000,2.900000,3.000000,3.000000,2.900000,3.100000
75%,4.100000,3.900000,3.900000,3.900000,4.000000,4.000000,3.900000,4.000000,3.900000,4.100000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


#Data Normalization

In [5]:
data_norm = data.subtract(data.mean(axis=1), axis=0)
data_norm

,PolicyID1,PolicyID2,PolicyID3,PolicyID4,PolicyID5,PolicyID6,PolicyID7,PolicyID8,PolicyID9,PolicyID10
UserID,,,,,,,,,,
1,1.375000,NaN,-1.525000,NaN,-0.525000,NaN,NaN,NaN,0.675000,NaN
2,1.566667,0.266667,-1.733333,NaN,-1.633333,NaN,NaN,1.166667,NaN,0.366667
3,0.100000,NaN,-1.400000,NaN,-1.000000,0.000000,NaN,1.500000,0.600000,0.200000
4,0.457143,NaN,-0.942857,-0.542857,0.957143,NaN,0.957143,-0.242857,-0.642857,NaN
5,NaN,0.912500,-0.287500,-1.287500,0.912500,-0.787500,-0.887500,NaN,0.112500,1.312500
...,...,...,...,...,...,...,...,...,...,...
996,-1.271429,-0.171429,1.328571,-0.871429,-1.271429,NaN,0.528571,NaN,NaN,1.728571
997,0.222222,-0.377778,2.222222,-1.177778,-1.177778,NaN,0.922222,-0.477778,-0.977778,0.822222
998,0.428571,-0.271429,NaN,-1.171429,NaN,NaN,0.928571,0.828571,-0.471429,-0.271429


#Identify Similar User

In [6]:
user_similarity = data_norm.T.corr()
user_similarity

UserID,1,2,3,4,5,6,7,8,9,10,...,991,992,993,994,995,996,997,998,999,1000
UserID,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.949342,0.891629,0.326571,0.277350,-0.613936,-0.886845,NaN,-1.000000,-0.735422,...,-1.000000,-0.907482,-1.000000,-0.887868,-1.000000,-0.764265,-0.519635,1.000000,NaN,-1.000000
2,0.949342,1.000000,0.850968,0.122796,0.699127,-0.654682,-0.976477,-0.541557,-1.000000,-0.194381,...,-0.882877,-0.666162,0.643175,-0.278419,0.604707,-0.193124,-0.232332,0.841403,1.000000,0.622164
3,0.891629,0.850968,1.000000,-0.096854,0.077254,-0.717485,-0.459785,-1.000000,-1.000000,-0.240403,...,-0.575930,-0.313312,0.484618,-0.179054,0.434803,-0.025728,-0.450158,0.552464,NaN,-0.306524
4,0.326571,0.122796,-0.096854,1.000000,0.270296,0.281077,-0.944445,-0.381246,-0.754260,-0.736376,...,-0.926665,-0.581887,-0.962950,-0.609396,-0.863315,-0.435165,-0.142152,0.733539,0.569495,-0.022354
5,0.277350,0.699127,0.077254,0.270296,1.000000,0.245871,-0.250863,0.385475,-0.804783,0.389131,...,0.606949,0.780197,-0.199260,0.494056,-0.979462,0.166920,-0.054078,0.020089,0.513418,0.615205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,-0.764265,-0.193124,-0.025728,-0.435165,0.166920,0.944789,0.792073,0.951325,0.708214,0.633520,...,0.145289,0.996059,0.673868,0.847575,1.000000,1.000000,0.788347,0.142269,0.538277,-0.190113
997,-0.519635,-0.232332,-0.450158,-0.142152,-0.054078,0.803103,0.373528,0.875168,0.611605,0.514509,...,0.224690,0.447941,0.461360,0.489673,0.534254,0.788347,1.000000,0.600461,0.391004,-0.044098
998,1.000000,0.841403,0.552464,0.733539,0.020089,0.049509,-0.754370,0.283877,-1.000000,-0.442261,...,-0.893941,-0.439757,0.240192,-0.423014,NaN,0.142269,0.600461,1.000000,0.271771,-0.019542


In [7]:
#Cosine similarity does not take missing value, we need to fill them with 0
user_similarity_cosine = cosine_similarity(data_norm.fillna(0))
user_similarity_cosine

array([[ 1.        ,  0.81658826,  0.60733468, ...,  0.06538198,
         0.06974401, -0.0620927 ],
       [ 0.81658826,  1.        ,  0.81833704, ...,  0.25265102,
         0.22704973,  0.11609906],
       [ 0.60733468,  0.81833704,  1.        , ...,  0.21466889,
         0.12464429, -0.04001103],
       ...,
       [ 0.06538198,  0.25265102,  0.21466889, ...,  1.        ,
         0.12856499, -0.07063408],
       [ 0.06974401,  0.22704973,  0.12464429, ...,  0.12856499,
         1.        ,  0.83971606],
       [-0.0620927 ,  0.11609906, -0.04001103, ..., -0.07063408,
         0.83971606,  1.        ]])

#Use userID1 as an example to illustrate how to find similar users.

In [8]:
#Pick a userID
Picked_UserID = 1

#Remove the picked UserID from the candidate list
user_similarity.drop(index=Picked_UserID, inplace=True)

user_similarity

UserID,1,2,3,4,5,6,7,8,9,10,...,991,992,993,994,995,996,997,998,999,1000
UserID,,,,,,,,,,,,,,,,,,,,,
2,0.949342,1.000000,0.850968,0.122796,0.699127,-0.654682,-0.976477,-0.541557,-1.000000,-0.194381,...,-0.882877,-0.666162,0.643175,-0.278419,0.604707,-0.193124,-0.232332,0.841403,1.000000,0.622164
3,0.891629,0.850968,1.000000,-0.096854,0.077254,-0.717485,-0.459785,-1.000000,-1.000000,-0.240403,...,-0.575930,-0.313312,0.484618,-0.179054,0.434803,-0.025728,-0.450158,0.552464,NaN,-0.306524
4,0.326571,0.122796,-0.096854,1.000000,0.270296,0.281077,-0.944445,-0.381246,-0.754260,-0.736376,...,-0.926665,-0.581887,-0.962950,-0.609396,-0.863315,-0.435165,-0.142152,0.733539,0.569495,-0.022354
5,0.277350,0.699127,0.077254,0.270296,1.000000,0.245871,-0.250863,0.385475,-0.804783,0.389131,...,0.606949,0.780197,-0.199260,0.494056,-0.979462,0.166920,-0.054078,0.020089,0.513418,0.615205
6,-0.613936,-0.654682,-0.717485,0.281077,0.245871,1.000000,-0.314570,0.922062,1.000000,0.451420,...,0.367020,0.479032,0.188982,0.505326,-0.593402,0.944789,0.803103,0.049509,0.662849,0.509313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,-0.764265,-0.193124,-0.025728,-0.435165,0.166920,0.944789,0.792073,0.951325,0.708214,0.633520,...,0.145289,0.996059,0.673868,0.847575,1.000000,1.000000,0.788347,0.142269,0.538277,-0.190113
997,-0.519635,-0.232332,-0.450158,-0.142152,-0.054078,0.803103,0.373528,0.875168,0.611605,0.514509,...,0.224690,0.447941,0.461360,0.489673,0.534254,0.788347,1.000000,0.600461,0.391004,-0.044098
998,1.000000,0.841403,0.552464,0.733539,0.020089,0.049509,-0.754370,0.283877,-1.000000,-0.442261,...,-0.893941,-0.439757,0.240192,-0.423014,NaN,0.142269,0.600461,1.000000,0.271771,-0.019542


#Top 10 most silimar users for UserID1

In [9]:
#Number of similar Users
n = 200

#User similarity Threshold
user_similarity_threshold = 0.3

#Get top n similar users
Similar_Users = user_similarity[user_similarity[Picked_UserID]>user_similarity_threshold][Picked_UserID].sort_values(ascending=False).head(n)

#Print out top n similar users
print(f'The similar users for user {Picked_UserID} are {Similar_Users}')

The similar users for user 1 are UserID
635    1.000000
277    1.000000
456    1.000000
69     1.000000
938    1.000000
         ...   
556    0.958187
248    0.954261
426    0.951613
544    0.950801
2      0.949342
Name: 1, Length: 200, dtype: float64


#Narrow Down Item Pool

In [10]:
#Policy that the target user has applied
Picked_UserID_Applied = data_norm[data_norm.index == Picked_UserID].dropna(axis=1, how='all')
Picked_UserID_Applied

,PolicyID1,PolicyID3,PolicyID5,PolicyID9
UserID,,,,
1,1.375,-1.525,-0.525,0.675


In [11]:
#Remove the policies the similar users have applied
Similar_Users_Applied = data_norm[data_norm.index.isin(Similar_Users.index)].dropna(axis=1, how='all')
Similar_Users_Applied

,PolicyID1,PolicyID2,PolicyID3,PolicyID4,PolicyID5,PolicyID6,PolicyID7,PolicyID8,PolicyID9,PolicyID10
UserID,,,,,,,,,,
2,1.566667,0.266667,-1.733333,NaN,-1.633333,NaN,NaN,1.166667,NaN,0.366667
19,NaN,-0.328571,-1.228571,NaN,-0.428571,0.471429,-1.528571,1.371429,1.671429,NaN
22,NaN,NaN,-1.420000,1.180000,NaN,NaN,-0.220000,NaN,1.280000,-0.820000
34,NaN,0.437500,-0.262500,-1.762500,0.637500,NaN,-0.962500,0.737500,1.237500,-0.062500
39,NaN,NaN,-1.480000,1.020000,NaN,NaN,NaN,-0.380000,0.920000,-0.080000
...,...,...,...,...,...,...,...,...,...,...
984,0.312500,0.712500,-2.287500,-0.387500,-1.287500,1.712500,NaN,NaN,-0.287500,1.512500
986,NaN,NaN,NaN,NaN,-0.780000,-0.080000,NaN,-0.780000,1.320000,0.320000
987,0.833333,0.233333,NaN,0.333333,NaN,0.033333,-0.666667,NaN,-0.766667,NaN


In [12]:
#Remove the applied policy from the policy list
Similar_Users_Applied.drop(Picked_UserID_Applied.columns, axis=1, inplace=True, errors='ignore')
Similar_Users_Applied

,PolicyID2,PolicyID4,PolicyID6,PolicyID7,PolicyID8,PolicyID10
UserID,,,,,,
2,0.266667,NaN,NaN,NaN,1.166667,0.366667
19,-0.328571,NaN,0.471429,-1.528571,1.371429,NaN
22,NaN,1.180000,NaN,-0.220000,NaN,-0.820000
34,0.437500,-1.762500,NaN,-0.962500,0.737500,-0.062500
39,NaN,1.020000,NaN,NaN,-0.380000,-0.080000
...,...,...,...,...,...,...
984,0.712500,-0.387500,1.712500,NaN,NaN,1.512500
986,NaN,NaN,-0.080000,NaN,-0.780000,0.320000
987,0.233333,0.333333,0.033333,-0.666667,NaN,NaN


#Recommended Policy

In [16]:
# Assuming Similar_Users_Applied is a DataFrame and Similar_Users is a Series

# Create a dictionary to store policies score
Policies_Score = {}

# Loop through policies
for i in Similar_Users_Applied.columns:
    # Get the rating from each policy
    policies_rating = Similar_Users_Applied[i]

    # Initialize variables to store the score and count
    total = 0
    count = 0

    # Loop through similar users
    for u in Similar_Users.index:
        # Check if the rating is not NaN
        if not pd.isna(policies_rating.loc[u]):
            score = Similar_Users[u] * policies_rating.loc[u]
            # Add the score to the total score for the policy
            total += score
            # Add 1 to the count
            count += 1

    # Calculate the average score for the policy if count is not zero
    if count > 0:
        Policies_Score[i] = total / count

# Convert dictionary to pandas dataframe
Policies_Score_df = pd.DataFrame(Policies_Score.items(), columns=['Policy', 'Policy_Score'])

# Sort the policy by score
Rank_Policy_Score = Policies_Score_df.sort_values(by='Policy_Score', ascending=False)

# Select top m policies
m = 10
Top_Policies = Rank_Policy_Score.head(m)
Top_Policies

,Policy,Policy_Score
3,PolicyID7,0.129467
4,PolicyID8,0.036961
0,PolicyID2,0.035847
2,PolicyID6,-0.006487
5,PolicyID10,-0.036470
1,PolicyID4,-0.129512
